# Script for CHM data preprocessing; adapted for inference map generation

## Imports and Paths definitons
watch out: data is not pushed to git. check that you have downloaded the raw data in the folder structure and named accordingly 

In [14]:
import os
import re
import subprocess
from collections import defaultdict
import numpy as np
import rasterio
from rasterio.merge import merge
from rasterio.warp import reproject, Resampling
import matplotlib.pyplot as plt
import glob
from rasterio.transform import Affine
from rasterio.warp import transform_bounds
from rasterio.warp import transform

#import geopandas as gpd
import pandas as pd

import sys
sys.path.append('../')

from utils import *
from utils import basics as bsc
from utils import processing as proc
import utils.config_loader as cf
import importlib
importlib.reload(bsc)

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
sites = cf.load_yaml("../configs/datapaths.yaml")

the subsequent pipelines expect a CHM for all preprocessing steps. we create one with all filled with 0. 
here we also specify the spatial extend, we want for the inference map.

In [16]:
ALS_PATH4_P = proc.get_pseudo_chm(sites, upper_left = (48.160723517168684, 11.55610203339669), lower_right = (48.11186967900207, 11.607257291939503),sitename="04_Munich", site_idx="04")
ALS_PATH5_P = proc.get_pseudo_chm(sites, upper_left = (47.750414329886446, 11.724883318084023), lower_right = (47.70782884189152, 11.888043785557002), sitename="05_Oberland", site_idx="05")

In [18]:
ALS_PATH4_RES = ALS_PATH4_P # already resampled, needed later
ALS_PATH5_RES = ALS_PATH5_P

In [21]:
#test: print the ground sampling distance of the generated pseudo-CHM
pseudo_chm_path = "../data/01_input_pipeline/04_Munich/pseudo_CHM.tif"
with rasterio.open(pseudo_chm_path) as src:
    transform = src.transform
    pixel_width = transform.a
    pixel_height = -transform.e  # negative because of the coordinate system
    print(f"Pixel Size: {pixel_width} x {pixel_height} units")

Pixel Size: 10.000402900886382 x 9.994089834515366 units


In [4]:
# PATH - Definitions
# === INPUT Paths ===

PATH_SITE1 = '../data/01_input_pipeline/01_Ebrach' 
PATH_SITE2 = '../data/01_input_pipeline/02_Wald'
PATH_SITE3 = '../data/01_input_pipeline/03_Berch'
PATH_SITE4 = '../data/01_input_pipeline/04_Munich'
PATH_SITE5 = '../data/01_input_pipeline/05_Oberland'

PATH_SITE1F = '../data/01_input_pipeline/01_Ebrach/01_' 
PATH_SITE2F = '../data/01_input_pipeline/02_Wald/02_'
PATH_SITE3F = '../data/01_input_pipeline/03_Berch/03_'
PATH_SITE4F = '../data/01_input_pipeline/04_Munich/04_'
PATH_SITE5F = '../data/01_input_pipeline/05_Oberland/05_'
PATH_S2_0102 = '../data/00_download_zips/S2_SEASONS/0102_Ebrach_Wald' # they're both in one folder
PATH_S2_03 = '../data/00_download_zips/S2_SEASONS/03_Berch'
PATH_S2_04 = '../data/00_download_zips/S2_SEASONS/04_MUC_OL'
PATH_S2_05 =  '../data/00_download_zips/S2_SEASONS/04_MUC_OL'

SITE_INDEX_EBRACH = '01_'
SITE_INDEX_WALD = '02_'
SITE_INDEX_BERCH = '03_'
SITE_INDEX_MUNICH = '04_'
SITE_INDEX_OBERLAND = '05_'

ALS_SUFFIX = 'ALS.tif'  # 'ALS_GT' or 'ALS_GT_Processed'
DEM_SUFFIX = 'DEM30.tif'  # 'DEM' or 'DEM_Processed'
DLT_SUFFIX = 'DLT.tif'  # 'DLTM' or 'DLTM_Processed'
FMASK_SUFFIX = 'FMA.tif'  # 'ForestMask' or 'ForestMask_Processed'

PATH_S2_Test1 = '../data/00_download_zips/S2_SEASONS/0102_Ebrach_Wald/summer/X0008_Y0004/20240101-20241231_152-243_HL_TSA_SEN2L_GRN_STM.tif'
PATH_S2_Test2 = '../data/00_download_zips/S2_SEASONS/03_Berch/spring/X0011_Y0007/20210101-20211231_060-151_HL_TSA_SEN2L_EVI_STM.tif'


## Sentinel-2 data ingest and pre-processing

### Conversion to .vrt in order to handle each band nicely

In [ ]:
# === CONFIG ===
recursive = True

proc.create_s2_vrt(PATH_S2_04, PATH_S2_04 + '/S2VRT/', recursive)
proc.create_s2_vrt(PATH_S2_05, PATH_S2_05 + '/S2VRT/', recursive)

Creating NDV_STM VRT for spring with 2 files...
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating RED_STM VRT for spring with 2 files...
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating EVI_STM VRT for spring with 2 files...
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating RE2_STM VRT for spring with 2 files...
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating BNR_STM VRT for spring with 2 files...
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating SW2_STM VRT for spring with 2 files...
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating BLU_STM VRT for spring with 2 files...
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating NBR_STM VRT for spring with 2 files...
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating RE3_STM VRT for spring with 2 files...
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating NIR_STM VR

### Align the S2 to ALS data

how to do alignment:
- first, we have to check for the same Map Projection.
- alignment and cropping:
    - polygon cutline around the images: downside: is very memory heavy (30GB)
    - instead, we crop an outer bounding box over the overlapping area for each ALS Site. 
- then, later we can drop parts where not all layers have plausible pixel values

### check projections

In [ ]:
VRT_FOLDER = PATH_S2_04 + '/S2VRT/spring/'    
proc.compare_raster_metadata(ALS_PATH4_P, os.path.join(VRT_FOLDER, "RED_STM.vrt"))


--- Raster A: 04_CHM.tif ---
Descriptions: (None,)
Shape: (530, 399)
Band count: 1
CRS: EPSG:25832
Transform: | 10.00, 0.00, 690073.84|
| 0.00,-9.99, 5337323.97|
| 0.00, 0.00, 1.00|

--- Raster B: RED_STM.vrt ---
Descriptions: ('Q50',)
Shape: (12000, 6000)
Band count: 1
CRS: EPSG:25832
Transform: | 10.00, 0.00, 688279.44|
| 0.00,-10.00, 5358053.30|
| 0.00, 0.00, 1.00|

--- Comparison ---
✅ CRS Projections match!
❌ Transformations do not match!


**🆒, both share the same coordinate system. the two diff. transformation, we'll take of that later when doing the resampling.**

### bounding box cropping around ALS dims

In [ ]:
# site 4:
proc.crop_and_stack_vrts_to_als(ALS_PATH4_P, PATH_S2_04 + '/S2VRT/', PATH_SITE4 + '/S2/', override=True)
# site 5:
proc.crop_and_stack_vrts_to_als(ALS_PATH5_P, PATH_S2_05 + '/S2VRT/', PATH_SITE5 + '/S2/', override=True)

[DEBUG] ALS bounds: BoundingBox(left=690073.838847652, bottom=5332027.104101895, right=694063.9996051056, top=5337323.971714188)
[DEBUG] ALS CRS: EPSG:25832
[DEBUG] VRT folder: ../data/00_download_zips/S2_SEASONS/04_MUC_OL/S2VRT/
[DEBUG] Output folder: ../data/01_input_pipeline/04_Munich/S2/
[WARP] Cropping RED_STM for spring (site 04_CHM.tif)...
[WARP] Cropping EVI_STM for spring (site 04_CHM.tif)...
[WARP] Cropping NDV_STM for spring (site 04_CHM.tif)...
[WARP] Cropping GRN_STM for spring (site 04_CHM.tif)...
[WARP] Cropping SW1_STM for spring (site 04_CHM.tif)...
[WARP] Cropping RE1_STM for spring (site 04_CHM.tif)...
[WARP] Cropping NBR_STM for spring (site 04_CHM.tif)...
[WARP] Cropping BLU_STM for spring (site 04_CHM.tif)...
[WARP] Cropping NIR_STM for spring (site 04_CHM.tif)...
[WARP] Cropping RE3_STM for spring (site 04_CHM.tif)...
[WARP] Cropping SW2_STM for spring (site 04_CHM.tif)...
[WARP] Cropping BNR_STM for spring (site 04_CHM.tif)...
[WARP] Cropping RE2_STM for spring 

### stack the cropped S2 back into one .tif with 13 channels x 5 stats = 65 channels per season

In [ ]:
# watch out: extend quantiles = True, because provided S2 stacks for inference were lacking the pixel quantiles!
# this is a hacky way to just extend them to the expected shape. the single median quantile is writting to all other channels.
# subsequent pipelines expects this shape 

S2Stack04 = proc.stack_site_vrts_to_multiband(PATH_SITE4 + '/S2/',override=True, extend_quantiles=True)
S2Stack05 = proc.stack_site_vrts_to_multiband(PATH_SITE5 + '/S2/',override=True, extend_quantiles=True)

✅ Stacked 13 bands with 5 channels each into ../data/01_input_pipeline/04_Munich/S2/spring/S2_stacked.tif
✅ Stacked 13 bands with 5 channels each into ../data/01_input_pipeline/04_Munich/S2/summer/S2_stacked.tif
✅ Stacked 13 bands with 5 channels each into ../data/01_input_pipeline/04_Munich/S2/autumn/S2_stacked.tif
✅ Stacked 13 bands with 5 channels each into ../data/01_input_pipeline/05_Oberland/S2/spring/S2_stacked.tif
✅ Stacked 13 bands with 5 channels each into ../data/01_input_pipeline/05_Oberland/S2/summer/S2_stacked.tif
✅ Stacked 13 bands with 5 channels each into ../data/01_input_pipeline/05_Oberland/S2/autumn/S2_stacked.tif


### little sanity check:

In [56]:
# test if band names are correct
with rasterio.open(S2Stack04['summer']) as s2_src:
    s2np = s2_src.read().astype(np.float32)  # shape: (bands, height, width)
    s2res_x, s2res_y = s2_src.res  # (pixel width, pixel height in coordinate units)
    s2crs = s2_src.crs
    # Print band (channel) descriptions if available
    print(s2_src.descriptions)

('BLU_Q50', 'BLU_Q50_a', 'BLU_Q50_b', 'BLU_Q50_c', 'BLU_Q50_d', 'BNR_Q50', 'BNR_Q50_a', 'BNR_Q50_b', 'BNR_Q50_c', 'BNR_Q50_d', 'EVI_Q50', 'EVI_Q50_a', 'EVI_Q50_b', 'EVI_Q50_c', 'EVI_Q50_d', 'GRN_Q50', 'GRN_Q50_a', 'GRN_Q50_b', 'GRN_Q50_c', 'GRN_Q50_d', 'NBR_Q50', 'NBR_Q50_a', 'NBR_Q50_b', 'NBR_Q50_c', 'NBR_Q50_d', 'NDV_Q50', 'NDV_Q50_a', 'NDV_Q50_b', 'NDV_Q50_c', 'NDV_Q50_d', 'NIR_Q50', 'NIR_Q50_a', 'NIR_Q50_b', 'NIR_Q50_c', 'NIR_Q50_d', 'RE1_Q50', 'RE1_Q50_a', 'RE1_Q50_b', 'RE1_Q50_c', 'RE1_Q50_d', 'RE2_Q50', 'RE2_Q50_a', 'RE2_Q50_b', 'RE2_Q50_c', 'RE2_Q50_d', 'RE3_Q50', 'RE3_Q50_a', 'RE3_Q50_b', 'RE3_Q50_c', 'RE3_Q50_d', 'RED_Q50', 'RED_Q50_a', 'RED_Q50_b', 'RED_Q50_c', 'RED_Q50_d', 'SW1_Q50', 'SW1_Q50_a', 'SW1_Q50_b', 'SW1_Q50_c', 'SW1_Q50_d', 'SW2_Q50', 'SW2_Q50_a', 'SW2_Q50_b', 'SW2_Q50_c', 'SW2_Q50_d')


## Now, we're ready for resampling
--> No need for resampling ALS, we have no raw chm 

## Aux Layer Transformations

### FMASK Transform

In [ ]:
proc.compare_raster_metadata(ALS_PATH4_RES, os.path.join(PATH_SITE4F + FMASK_SUFFIX))
proc.compare_raster_metadata(ALS_PATH5_RES, os.path.join(PATH_SITE5F + FMASK_SUFFIX))

In [ ]:
fma4p, meta, PATH_FMA4_P = proc.transform_and_crop_to_als(PATH_SITE4F + FMASK_SUFFIX, ALS_PATH4_RES, S2Stack04['summer'], PATH_SITE4)
fma5p, meta, PATH_FMA5_P = proc.transform_and_crop_to_als(PATH_SITE5F + FMASK_SUFFIX, ALS_PATH5_RES, S2Stack05['summer'], PATH_SITE5)

### DLT Transform (optional)
TO-DO: also adapt DLT

In [ ]:
# proc.compare_raster_metadata(PATH_SITE4F, os.path.join(ALS_PATH4_RES + DLT_SUFFIX))
# proc.compare_raster_metadata(PATH_SITE5F, os.path.join(ALS_PATH5_RES + DLT_SUFFIX))


In [ ]:
# dlt1p, meta, PATH_DLT1_P = proc.transform_and_crop_to_als(PATH_SITE1F + DLT_SUFFIX, ALS_PATH1_RES, S2Stack01['summer'], PATH_SITE1)
# dlt2p, meta, PATH_DLT2_P = proc.transform_and_crop_to_als(PATH_SITE2F + DLT_SUFFIX, ALS_PATH2_RES, S2Stack02['summer'], PATH_SITE2)
# dlt3p, meta, PATH_DLT3_P = proc.transform_and_crop_to_als(PATH_SITE3F + DLT_SUFFIX, ALS_PATH3_RES, S2Stack03['summer'], PATH_SITE3)

In [ ]:
# proc.compare_raster_metadata(ALS_PATH2_RES, S2Stack02['summer'])
# proc.compare_raster_metadata(ALS_PATH2_RES, PATH_DLT2_P)

### DEM Transform (optional)
TO-DO: also adapt DEM

In [ ]:
# proc.compare_raster_metadata(ALS_PATH2_RES, os.path.join(PATH_SITE2F + DEM_SUFFIX))

In [ ]:
# custom function to resample and crop DEM30 to ALS grid because we need 10m resolution DEM
import os

def resample_and_crop_dem_to_als(dem30_path, als_ref_path, output_folder, resampling_method=Resampling.bilinear):
    """
    Resample DEM from 30m to 10m resolution and crop to ALS reference raster.
    Output filename will be similar to input, but with 'DEM10' instead of 'DEM30'.
    """
    # Generate output filename
    dem30_filename = os.path.basename(dem30_path)
    dem10_filename = dem30_filename.replace('DEM30', 'DEM10')
    output_path = os.path.join(output_folder, dem10_filename)

    # Open ALS reference to get target shape, transform, crs
    with rasterio.open(als_ref_path) as als_src:
        target_crs = als_src.crs
        target_transform = als_src.transform
        target_height = als_src.height
        target_width = als_src.width
        target_meta = als_src.meta.copy()
        target_meta.update({
            'driver': 'GTiff',
            'dtype': 'float32',
            'compress': 'lzw',
            'tiled': True,
            'blockxsize': 256,
            'blockysize': 256,
            'interleave': 'band',
            'count': 1,
            'nodata': 0
        })

    # Open DEM30 and reproject/resample to ALS grid
    with rasterio.open(dem30_path) as dem_src:
        dem_data = dem_src.read(1)
        dem10 = np.zeros((target_height, target_width), dtype=np.float32)
        reproject(
            source=dem_data,
            destination=dem10,
            src_transform=dem_src.transform,
            src_crs=dem_src.crs,
            dst_transform=target_transform,
            dst_crs=target_crs,
            resampling=resampling_method
        )


    # Write output
    with rasterio.open(output_path, 'w', **target_meta) as dst:
        dst.write(dem10, 1)

    return dem10, target_meta, output_path


In [ ]:
# dem1p, meta, PATH_DEM1_P = proc.resample_and_crop_dem_to_als(PATH_SITE1F + DEM_SUFFIX, ALS_PATH1_RES, PATH_SITE1)
# dem2p, meta, PATH_DEM2_P = proc.resample_and_crop_dem_to_als(PATH_SITE2F + DEM_SUFFIX, ALS_PATH2_RES, PATH_SITE2)
# dem3p, meta, PATH_DEM3_P = proc.resample_and_crop_dem_to_als(PATH_SITE3F + DEM_SUFFIX, ALS_PATH3_RES, PATH_SITE3)

In [ ]:
# proc.compare_raster_metadata(ALS_PATH2_RES, PATH_DEM2_P)

# That's it for all resampling and cropping!